# Tutorial for using PMO database

We illustrate briefly the main functionality of the package `PMO.jl` dedicated to Polynomial & Moment Optimization problems. 

## Polynomial data

Polynomial optimization problems are of the form 
$$
\begin{array}{rl}
\inf_{x\in \mathbb{R}^n} & f(x)\\ 
 s.t. & g_1(x)\ge 0, \ldots, g_s(x) \geq 0\\
      & h_1(x) = 0, \ldots, h_t(x) =0
\end{array}
$$
where $f, g_1, \ldots, g_s, h_1, ..., h_t \in \mathbb{R}[x_1, \ldots, x_n]$.

 * $f$ is the objective function,
 * $g_1, \ldots, g_s$ are sign constraints,
 * $h_1, ..., h_t$ are equality constraints.

In [1]:
using PMO, DynamicPolynomials

X = @polyvar x y

f  = x^2*y^2+x^4-y^3
g1 = x^2 + Float64(pi)*y^2 -2
g2 = x

h1 = 2*y^2-y


F  = PMO.polynomial((f,"inf"),
                    (g1,"<=0"),
                    (g2,">=0"),
                    (h1,"=0"))

F["doc"]= 
"""
A first polynomial example with two sign constraints and one equality constraint.
"""

┌ Info: Precompiling PMO [fbf10557-b722-42fa-b9dc-91a07c0260e4]
└ @ Base loading.jl:1273


"A first polynomial example with two sign constraints and one equality constraint.\n"

Sign constraints  can be of the form `"<=0"` and `">=0"`. Notice that `">0"`or `"<0"` are **not** handled.  

The same data `F` could also have been defined as follows:

In [2]:
F  = PMO.data((f,"inf"),
              (g1,"<=0"),
              (g2,">=0"),
              (h1,"=0") ; type = "polynomial")

Optimisation model:
  type => polynomial
  variables => ["x", "y"]
  nvar => 2
  constraints => [ x^2 + 3.141592653589793*y^2 - 2.0 <=0, x >=0, 2*y^2 - y =0 ]
  objective => inf x^4 + x^2*y^2 - y^3
  version => 0.0.1
  uuid => da37654c-9154-11eb-16a1-afb8fe4392d1


This object of type `PMOData` can be printed and saved in json format as follows:

In [3]:
PMO.write(F)
PMO.write("tmp.json",F)

{
  "type": "polynomial",
  "variables": [
    "x",
    "y"
  ],
  "nvar": 2,
  "constraints": [
    {
      "set": "<=0",
      "polynomial": {
        "coeftype": "Float64",
        "terms": [
          [1.0,[2],[1]],
          [3.141592653589793,[2],[2]],
          [-2.0]
        ]
      }
    },
    {
      "set": ">=0",
      "polynomial": {
        "coeftype": "Int64",
        "terms": [
          [1,[1],[1]]
        ]
      }
    },
    {
      "set": "=0",
      "polynomial": {
        "coeftype": "Int64",
        "terms": [
          [2,[2],[2]],
          [-1,[1],[2]]
        ]
      }
    }
  ],
  "objective": {
    "set": "inf",
    "polynomial": {
      "coeftype": "Int64",
      "terms": [
        [1,[4],[1]],
        [1,[2,2]],
        [-1,[3],[2]]
      ]
    }
  },
  "version": "0.0.1",
  "uuid": "da37654c-9154-11eb-16a1-afb8fe4392d1"
}


We read the data saved in the file `tmp.json` and built the corresponding data `G` of type `PMOData`, which `json` format is the same as the one of `F`.

In [4]:
G  = PMO.read("tmp.json")
PMO.write(G)

{
  "type": "polynomial",
  "variables": [
    "x",
    "y"
  ],
  "nvar": 2,
  "constraints": [
    {
      "set": "<=0",
      "polynomial": {
        "coeftype": "Float64",
        "terms": [
          [1.0,[2],[1]],
          [3.141592653589793,[2],[2]],
          [-2.0]
        ]
      }
    },
    {
      "set": ">=0",
      "polynomial": {
        "coeftype": "Int64",
        "terms": [
          [1,[1],[1]]
        ]
      }
    },
    {
      "set": "=0",
      "polynomial": {
        "coeftype": "Int64",
        "terms": [
          [2,[2],[2]],
          [-1,[1],[2]]
        ]
      }
    }
  ],
  "objective": {
    "set": "inf",
    "polynomial": {
      "coeftype": "Int64",
      "terms": [
        [1,[4],[1]],
        [1,[2,2]],
        [-1,[3],[2]]
      ]
    }
  },
  "version": "0.0.1",
  "uuid": "da37654c-9154-11eb-16a1-afb8fe4392d1"
}


## Moment data

Moment optimization problems are of the form 
$$
\begin{array}{rl}
\inf_{\mu_j\in \mathbb{R}[x_1, \ldots, x_n]^* } & \langle\mu_1|f_1\rangle+\cdots +\langle\mu_\nu|f_\nu\rangle\\ 
 s.t. & \sum_{j=1}^{\nu} g_{1,j} \star \mu_j \succeq 0, \ldots, \sum_{j=1}^{\nu} g_{s,j} \star \mu_j \succeq 0\\
      & \sum_{j=1}^{\nu} h_{1,j} \star \mu_j = 0, \ldots, \sum_{j=1}^{\nu} h_{t,j} \star \mu_j =0\\
      & \sum_{j=1}^{\nu} \langle\mu_j|l_{1,j}\rangle\ge 0, \ldots, \sum_{j=1}^{\nu}\langle\mu_j|l_{u,j}\rangle\ge 0\\
      & \sum_{j=1}^{\nu} \langle\mu_j|m_{1,j}\rangle = 0, \ldots,  \sum_{j=1}^{\nu}\langle\mu_j|l_{v,j}\rangle = 0
\end{array}
$$
where
  * $\mu_1, \ldots, \mu_\nu$ are moment sequences, i.e. elements of the dual $\mathbb{R}[x_1, \ldots, x_n]^*$,
  * $f, g_{i,j}, h_{i,j}, l_{i,j}, m_{i,j}\in \mathbb{R}[x_1, \ldots, x_n]$.

The constraints $\sum_{j=1}^{\nu} \langle\mu_i|l_{i,j}\rangle\ge 0$, $\sum_{j=1}^{\nu} \langle\mu_j|m_{i,j}\rangle = 0$ are respectively mass sign constraints and mass equality constraints.

The product $\star$ is defined as follows: for $p\in \mathbb{R}[x_1, \ldots, x_n], \mu \in \mathbb{R}[x_1, \ldots, x_n]^*$, $p\star \mu : q \mapsto \langle p\star \mu|q\rangle:= \langle\mu| p q\rangle$.  

In [5]:
using DynamicPolynomials, PMO
@polyvar x y
f1 = x^2*y^2+x^4-y^3
f2 = x*y

g1  = x^2 + Float64(pi)*y^2 -2
g2 = x

h1 = 2*y^2-y
h2 = x^2+y*2.1*x*y

F  = PMO.moment(([f1,f2],"inf"),
                ([g1,0],">=0"),
                ([0,g2], ">=0"),
                ([h1, h2], "=0 *")
                )

Optimisation model:
  type => moment
  variables => ["x", "y"]
  nvar => 2
  nms => 2
  constraints => [ Polynomial{true,Float64}[x² + 3.141592653589793y² - 2.0, 0.0] * >=0, Polynomial{true,Int64}[0, x] * >=0, Polynomial{true,Float64}[2.0y² - y, 2.1xy² + x²] * =0 * ]
  objective => inf Polynomial{true,Int64}[x⁴ + x²y² - y³, xy]
  version => 0.0.1
  uuid => dc815b50-9154-11eb-3dca-93acb42c6bb6


In [6]:
PMO.write("tmp.json",F)
G  = PMO.read("tmp.json")
PMO.write(G)

{
  "type": "moment",
  "variables": [
    "x",
    "y"
  ],
  "nvar": 2,
  "nms": 2,
  "constraints": [
    {
      "set": ">=0",
      "moments": {
        "coeftype": "Float64",
        "terms": [
          [1.0,1,[2],[1]],
          [3.141592653589793,1,[2],[2]],
          [-2.0,1]
        ]
      }
    },
    {
      "set": ">=0",
      "moments": {
        "coeftype": "Int64",
        "terms": [
          [1,2,[1],[1]]
        ]
      }
    },
    {
      "set": "=0 *",
      "moments": {
        "coeftype": "Float64",
        "terms": [
          [2.0,1,[2],[2]],
          [-1.0,1,[1],[2]],
          [2.1,2,[1,2]],
          [1.0,2,[2],[1]]
        ]
      }
    }
  ],
  "objective": {
    "set": "inf",
    "moments": {
      "coeftype": "Int64",
      "terms": [
        [1,1,[4],[1]],
        [1,1,[2,2]],
        [-1,1,[3],[2]],
        [1,2,[1,1]]
      ]
    }
  },
  "version": "0.0.1",
  "uuid": "dc815b50-9154-11eb-3dca-93acb42c6bb6"
}


## SDP data

SDP optimization problems are of the form
$$
\begin{array}{rl}
\inf_{x\in \mathbb{R}^n} & f^T x \\ 
 s.t. & \sum_{j=1}^{n} A_{1,j} x_j + A_{1,0} \succeq 0, \ldots, \sum_{j=1}^{n} A_{s,j} x_j + A_{s,0} \succeq 0\\
      & \sum_{j=1}^{n} c_{1,1}^T x + d_{1,1}\ge 0, \ldots, \sum_{j=1}^{n} c_{1,u}^T x + d_{1,u}\ge 0\\
      & \sum_{j=1}^{n} c_{0,1}^T x + d_{0,1}\ge 0, \ldots, \sum_{j=1}^{n} c_{0,v}^T x + d_{0,v} = 0
\end{array}
$$
where $f, c_{i,j} \in \mathbb{R}^n$, $d_{i,j}\in \mathbb{R}$, $A_{i,j} \in S^{n_i}$ are symmetric matrices of size $n_i$.

In [7]:
using PMO, LinearAlgebra

LMI1 = [Symmetric([2 -1 0; 0 2 0; 0 0 2]),
        0,
        Symmetric([2 0 -1; 0 2 0; 0 0 2])
       ]

LMI2 = [Symmetric([1 0; 0 -1]),
        Symmetric([0 3; 3 0 ]),
        0,
        Symmetric([0 -1; -1 2])
       ]

F  = PMO.sdp(([1,2,3], "inf"),
             (LMI1,">=0"),
             (LMI2,">=0"),
             ([1.1,2,0,-4], "=0"),
             ([0,-1.2,3,-1],"<=0"),
             )
F[:name] = "My first example"
F[:doc]  = "Two linear matrix inequalities, one linear scalar equality and one linear scalar inequality."

"Two linear matrix inequalities, one linear scalar equality and one linear scalar inequality."

In [8]:
PMO.write("tmp.json",F)
G  = PMO.read("tmp.json")
PMO.write(G)

{
  "type": "sdp",
  "nvar": 3,
  "objective": [1,2,3],
  "constraints": {
    "nlmi": 2,
    "msizes": [3,2],
    "lmi_symat": [
      [2.0,1,1,1,1],
      [-1.0,1,1,2,1],
      [2.0,1,1,2,2],
      [2.0,1,1,3,3],
      [2.0,3,1,1,1],
      [2.0,3,1,2,2],
      [-1.0,3,1,3,1],
      [2.0,3,1,3,3],
      [1.0,1,2,1,1],
      [-1.0,1,2,2,2],
      [3.0,2,2,2,1],
      [-1.0,0,2,2,1],
      [2.0,0,2,2,2]
    ],
    "nlsi": 2,
    "lsi_mat": [
      [1.1,1,1],
      [2.0,1,2],
      [1.2,2,2],
      [-3.0,2,3]
    ],
    "lsi_vec": [4.0,-1.0],
    "lsi_op": [0,1]
  },
  "version": "0.0.1",
  "uuid": "dd6731ca-9154-11eb-05ba-a1de43b433cb",
  "name": "My first example",
  "doc": "Two linear matrix inequalities, one linear scalar equality and one linear scalar inequality."
}


In [9]:
LMI1 = [Symmetric([2 1 0; 0 1 0; 0 0 0]),
        Symmetric([0 0 0; 0 1 1; 0 0 1]),
        [[0,0,1]],
        -Symmetric([0 0 0; 0 0 1; 0 0 0])]

F  = PMO.sdp(([0,0,1], "inf"),
             (LMI1, ">=0"),
             ([1, 1, 0 , -1], "=0"),
             ([1, 0, 0 ], ">=0"),
             ([0, 1, 0 ], ">=0")
             )
F[:name] = "A second example"
F[:doc] =
    """
    one LMI with one rank-1 matrix, 3 linear scalar constraints
    """

"one LMI with one rank-1 matrix, 3 linear scalar constraints\n"

Here in `LMI1`, the third matrix is a low rank matrix, represented as a vector of vectors $u_i$ corresponding to $\sum_i u_i\otimes u_i$.

In [10]:
PMO.write("tmp.json",F)
G  = PMO.read("tmp.json")
PMO.write(G)

{
  "type": "sdp",
  "nvar": 3,
  "objective": [0,0,1],
  "constraints": {
    "nlmi": 1,
    "msizes": 3,
    "lmi_symat": [
      [2.0,1,1,1,1],
      [1.0,1,1,2,1],
      [1.0,1,1,2,2],
      [1.0,2,1,2,2],
      [1.0,2,1,3,2],
      [1.0,2,1,3,3],
      [-1.0,0,1,3,2]
    ],
    "lmi_lrmat": [
      [1.0,3,1,1,3]
    ],
    "nlsi": 3,
    "lsi_mat": [
      [1.0,1,1],
      [1.0,1,2],
      [1.0,2,1],
      [1.0,3,2]
    ],
    "lsi_vec": [1.0,0.0,0.0],
    "lsi_op": [0,1,1]
  },
  "version": "0.0.1",
  "uuid": "ddfb4c7a-9154-11eb-35f5-8bb85263cdd6",
  "name": "A second example",
  "doc": "one LMI with one rank-1 matrix, 3 linear scalar constraints\n"
}


## Registering and reading data

The PMO data base is loaded as follows:

In [11]:
using PMO
t = PMO.table()

PMO.DataBase(Table with 12 rows, 3 columns:
Columns:
#  colname  type
──────────────────
1  uuid     String
2  name     String
3  url      String)

It is a table of triplets `(uuid , name, url)` of a unique identifier `uuid`, a `name` containing tags to recover easily the data and the `url` of the file containing the data. This table only contains references to the PMO data. These data files are available in the local folder `$HOME/.julia/PMO/data`. 

## Selecting data

Data can be selected from their `:name` attribute, by regular expressions or matching strings.

In [12]:
t2 = select(t,r"[Mm]otz")
t3 = select(t,"Motzkin")

PMO.DataBase(Table with 3 rows, 3 columns:
Columns:
#  colname  type
──────────────────
1  uuid     String
2  name     String
3  url      String)

This gives new tables `t2, t3` of triplets `(uuid,name,url)` (Here with 3 rows). 

To select one column of such a table, one can do the following:

In [13]:
select(t2, :name)

3-element WeakRefStrings.StringArray{String,1}:
 "Motzkin"            
 "Motzkin homogeneous"
 "Motzkin bounded"    

We can recover the i$^{th}$ entry of the table as follows:

In [14]:
t2[1]

Optimisation model:
  type => polynomial
  variables => Any["x", "y"]
  nvar => 2
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
  name => Motzkin
  version => 0.0.1
  author => Bernard Mourrain
  uuid => cb46cb04-83db-11ea-0b43-87115082d076
  doc => Celebrate Motzkin polynomial, which is a non-negative polynomial, but not a Sum of Squares



Data can also be recovered from regular expressions matching their names:

In [15]:
t2[r"Motz."]

3-element Array{Any,1}:
 Optimisation model:
  type => polynomial
  variables => Any["x", "y"]
  nvar => 2
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
  name => Motzkin
  version => 0.0.1
  author => Bernard Mourrain
  uuid => cb46cb04-83db-11ea-0b43-87115082d076
  doc => Celebrate Motzkin polynomial, which is a non-negative polynomial, but not a Sum of Squares

                          
 Optimisation model:
  type => polynomial
  variables => Any["x", "y", "z"]
  nvar => 3
  constraints => [ x^2 + y^2 + z^2 - 1 =0 ]
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2*z^2 + z^6
  version => 0.0.2
  uuid => e46b1efe-0a05-11eb-198e-0912f9f7d742
  name => Motzkin homogeneous
  doc => Homogeneous Motzkin polynomial on the unit sphere

  author => Bernard Mourrain

 Optimisation model:
  type => polynomial
  variables => Any["x", "y"]
  nvar => 2
  constraints => [ -x^2 - y^2 + 2 >=0 ]
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
  version => 0.0.2
  uuid => dab9e7be-19fa-11e

or equivalently the elements, which name contains "Motz":

In [16]:
m = t3["Motz"]

3-element Array{PMO.Data,1}:
 Optimisation model:
  type => polynomial
  variables => Any["x", "y"]
  nvar => 2
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
  name => Motzkin
  version => 0.0.1
  author => Bernard Mourrain
  uuid => cb46cb04-83db-11ea-0b43-87115082d076
  doc => Celebrate Motzkin polynomial, which is a non-negative polynomial, but not a Sum of Squares

                          
 Optimisation model:
  type => polynomial
  variables => Any["x", "y", "z"]
  nvar => 3
  constraints => [ x^2 + y^2 + z^2 - 1 =0 ]
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2*z^2 + z^6
  version => 0.0.2
  uuid => e46b1efe-0a05-11eb-198e-0912f9f7d742
  name => Motzkin homogeneous
  doc => Homogeneous Motzkin polynomial on the unit sphere

  author => Bernard Mourrain

 Optimisation model:
  type => polynomial
  variables => Any["x", "y"]
  nvar => 2
  constraints => [ -x^2 - y^2 + 2 >=0 ]
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
  version => 0.0.2
  uuid => dab9e7be-19f

The PMO data can be transformed with `vec` into an array of pairs of polynomials and constraints that can be easily used in other functions: 

In [17]:
P = vec(t[1])

4-element Array{Any,1}:
 (x⁴ + x²y² - y³, "inf")                
 (x² + 3.141592653589793y² - 2.0, "<=0")
 (x, Any[-1, 1])                        
 (2y² - y, "=0")                        

Here is an illustration with the packages `MomentTools`, `MosekTools`, using the functions `vec` and `variables`:

In [18]:
 P = t[r"Motz.*bound"][1]

Optimisation model:
  type => polynomial
  variables => Any["x", "y"]
  nvar => 2
  constraints => [ -x^2 - y^2 + 2 >=0 ]
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
  version => 0.0.2
  uuid => dab9e7be-19fa-11eb-329b-c5570aaed8a0
  name => Motzkin bounded
  author => Bernard Mourrain


In [19]:
using MomentTools, MosekTools, JuMP
#optimizer = Mosek.Optimizer    #with verbose execution
optimizer = optimizer_with_attributes(Mosek.Optimizer, "QUIET"=>true)  #with option "quiet" for silent execution.
v, M = optimize(vec(P),variables(P), 3, optimizer)     #Moment relaxation at order 3
get_minimizers(M)

2×4 Array{Float64,2}:
 -0.999949  -0.999949   0.999949  0.999949
 -0.999949   0.999949  -0.999949  0.999949

## Updating the database

The database can be updated from the git server as follows:

In [20]:
update(t)

┌ Info: PMO update data
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:40


Table with 12 rows, 3 columns:
Columns:
#  colname  type
──────────────────
1  uuid     String
2  name     String
3  url      String

#### Adding new data

New data can be added as follows. First we create the data:

In [21]:
using PMO, DynamicPolynomials
X = @polyvar x y
motz = x^4*y^2 + x^2*y^4 + 1 - 3*x^2*y^2

Motz = PMO.data((motz,"inf"), (2-x^2-y^2, ">=0"))
Motz[:name] = "Motzkin bounded"

"Motzkin bounded"

Now we push it in the database with the file name `motzkin_bounded`:

In [22]:
t = PMO.table()
push(t, Motz, file="motzkin_bounded")

┌ Warning: data[:author] not defined; data not pushed to database
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:296


The name `motzkin_bounded` specifies the file used to store the data. 

Here we get a warning because the attribute `:author` is not defined in Motz. So we define it and retry:

In [23]:
Motz[:author] = "Joe Test"
push(t, Motz, file="motzkin_bounded")

┌ Info: PMO update data
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:40
┌ Info: PMO data motzkin_bounded.json already exists
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:116
┌ Info: PMO add data motzkin_bounded.1.json
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:129
┌ Info: PMO update registery
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:176
┌ Info: PMO update data
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:40
┌ Info: PMO update table
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:306


We see that the file name `motzkin_bounded` is already used so that the data is stored as the file `motzkin_bounded.1.json` in the folder `json`of the data project `PMO.PMO_GIT_DATA_URL`. The index table of the registry folder is updated with a line per data. A local version of the data bases and index files are updated; `git` command to commit and push the changes on the `git` projects.

Data can be removed from the database as follows:

In [24]:
PMO.rm(t, Motz)

┌ Info: PMO remove data motzkin_bounded.1.json
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:107


#### Modifying a data 

Here, it is showing how to modify an element and update of the database:

In [25]:
F = t[2]

F[:doc] = """
Celebrate Motzkin polynomial, which is a non-negative polynomial, but not a Sum of Squares
"""

"Celebrate Motzkin polynomial, which is a non-negative polynomial, but not a Sum of Squares\n"

To update of the database element of the table `t`:

In [26]:
push(t,F)

┌ Warning: same data; no commit and push
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:314


The data would modified localy and on the `git` server. Here, because it is the same text, there is a warning saying that no modification is commited on the `git`server.

In [27]:
t[2]

Optimisation model:
  type => polynomial
  variables => Any["x", "y"]
  nvar => 2
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
  name => Motzkin
  version => 0.0.1
  author => Bernard Mourrain
  uuid => cb46cb04-83db-11ea-0b43-87115082d076
  doc => Celebrate Motzkin polynomial, which is a non-negative polynomial, but not a Sum of Squares



#### Managing tables

Tables can be saved and loaded from files, as illustrated in the following example.

Suppose you want to work on all data related to `Motzkin`. First, you can select them from the PMO database and build a new database `tm`:

In [28]:
using PMO
t = PMO.table()
tm = select(t,"Motz")

PMO.DataBase(Table with 3 rows, 3 columns:
Columns:
#  colname  type
──────────────────
1  uuid     String
2  name     String
3  url      String)

Next you can save it in a file`motztable.csv` for futur use:

In [29]:
PMO.write("motztable.csv",tm)

Once saved in this file, the table can be reloaded without loading the complete PMO database, as follows:

In [30]:
ts = PMO.table("motztable.csv")

PMO.DataBase(Table with 3 rows, 3 columns:
Columns:
#  colname  type
──────────────────
1  uuid     String
2  name     String
3  url      String)